<a href="https://colab.research.google.com/github/toche7/AI_ITM/blob/main/Lab8Intro_PyTorch_toNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction to PyTorch**

**Install PyTorch:**



*   Make sure PyTorch is installed. You can install it via pip with the following command (you may need to adjust the command based on your system and whether you want GPU support):


In [ ]:
!pip install torch

**Tensors:**

*   Tensors are the core data structures in PyTorch. You can think of them as generalized arrays or matrices. Here’s how to create a tensor



In [ ]:
import torch

# Create a 1D tensor
tensor_1d = torch.tensor([1, 2, 3])
print(tensor_1d)

In [ ]:
# Create a 2D tensor (matrix)
tensor_2d = torch.tensor([[1, 2], [3, 4]])
print(tensor_2d)

**Basic Operations:**


*   You can perform various mathematical operations on tensors


In [ ]:
# Addition
result = tensor_1d + 2
print(result)

In [ ]:
# Matrix multiplication
tensor_a = torch.tensor([[1, 2], [3, 4]])
tensor_b = torch.tensor([[5, 6], [7, 8]])
result = torch.mm(tensor_a, tensor_b)
print(result)

**Autograd:**


*   PyTorch has a built-in automatic differentiation library. This is useful for backpropagation in neural networks.

In [ ]:
# Create a tensor with requires_grad=True to track operations.
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
z = y * y * 3

In [ ]:
# Compute gradients
z.backward(torch.ones_like(z))
print(x.grad)  # This will output the gradient of z with respect to x

**Creating and Training a Simple Neural Network:**

Here's a basic example of defining a neural network and training it.

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Define a simple feedforward network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2, 2)  # Input layer to hidden layer
        self.fc2 = nn.Linear(2, 1)  # Hidden layer to output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Instantiate the model, define the loss function and the optimizer
model = SimpleNN()
criterion = nn.MSELoss()  # Mean squared error loss
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Stochastic gradient descent

In [ ]:
# Example training loop
for epoch in range(100):
    # Dummy input and target
    input_data = torch.tensor([[1.0, 2.0]])
    target_data = torch.tensor([[1.0]])

    # Forward pass
    optimizer.zero_grad()
    output = model(input_data)
    loss = criterion(output, target_data)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

**Saving and Loading Models:**

After training a model, you might want to save it and load it later.

In [ ]:
# Save the model
torch.save(model.state_dict(), 'model.pth')

In [ ]:
# Load the model
model = SimpleNN()
model.load_state_dict(torch.load('model.pth'))
model.eval()  # Set the model to evaluation mode

# **PyTorch Introduction to Neual Network**

**Install PyTorch:**



*   Make sure PyTorch is installed. You can run the following command in a code cell to install PyTorch and TorchVision, which contains the MNIST dataset



In [ ]:
!pip install torch torchvision

**Import Necessary Libraries:**

*   Import PyTorch and other libraries required for your analysis



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

**Load the MNIST Dataset:**

*   Utilize torchvision.datasets to load the MNIST data



In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=True, download=True, transform=transform),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=False, transform=transform),
    batch_size=1000, shuffle=False)

**Define a Simple Neural Network:**

*   Create a simple PyTorch neural network




In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()

**Define Training and Test Functions:**

*   Set up functions to train and test the model:



In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss(reduction='sum')(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)')

**Train the Model:**



*   Finally, run the training and testing



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)